# Ансамбли моделей в машинном обучении

Техника **ансамблирования** (т. е. комбинирования разных моделей для создания одной "оптимальной") весьма популярна в машинном обучении. Это даёт возможность не полагаться на одну-единственную модель, которая может быть переобучена или иметь другие недостатки. С частным случаем ансамбля моделей мы уже сталкивались, когда рассматривали Random Forest. Теперь рассмотрим эту технику в более общей форме познакомимся с другими типами ансамблей.

В основном различают три вида ансамблей:
* бэггинг;
* бустинг;
* стекинг (блендинг).

## Бэггинг

Вспомним, что случайный лес (Random Forest) --- это ансамбль решающих деревьев вида **bagging (bootstrap aggregating)**. Суть в том, что мы обучаем решающие деревья на выборках, полученных из исходной обучающей выборки путём бутстрэпа. При этом каждое отдельное дерево может быть переобученным, но **после агрегирования** ансамбль даёт лучшие результаты, чем каждая его компонента в отдельности. Итак, бэггинг --- это "микс" из различных **однородных** моделей, обучаемых **параллельно и независимо** друг от друга (чем более независимо, тем лучше!). Конечный результат получается путём агрегирования (часто --- просто усреднения) результатов.

## Бустинг

**Бустинг (boosting)** моделей основан на другой идее. Предположим, у нас есть некоторая базовая (слабая) модель. Давайте **последовательно улучшать её качество** путём, например, анализа ошибок, которые модель допускала на предыдущей итерации. При этом каждая новая модель будет иметь лучший score, чем предыдущая. В бустинге также рассматриваются **однородные модели** (как и в бэггинге, популярным выбором являются деревья, но это вовсе не обязательно). Бустинг похож на итеративный процесс оптимизации параметров функции по градиентному спуску, только в данном случае мы "оптимизируем" саму модель в некотором "пространстве моделей".

## Стекинг

Наконец, **стекинг (stacked generalization, stacking)** --- это ансамбль, основанный на идее **добавления метапризнаков** к исходному набору признаков и **обучения метамодели** на наборе метапризнаков. Как правило, метапризнаки строятся как результаты предсказаний различных моделей ML. В очень грубой форме стекинг можно описать так: 
* Шаг 1. Строим кучу разных моделей на исходном наборе признаков (например, линейную регрессию, регрессию по knn, регрессию по деревьям, random forest и пару нейросетей впридачу) --- всё, на что способна бурная фантазия ML engineer :)
* Шаг 2. Каждая из моделей на шаге 1 дала какие-то предсказания, давайте добавим эти предсказания к исходному набору признаков. Получим так называемые "метапризнаки".
* Шаг 3. Обучим новую модель ML (например, опять линейную регрессию) на наборе метапризнаков. Такая модель называется "метамоделью". Результат метамодели будем считать окончательным предсказанием.

Описанный вид стекинга самый простой, его часто называют **"блендингом" (смешиванием)**. При желании можно делать многоуровневый стекинг, создавая "метамета...метапризнаки" и "метамета...метамодели" описанным способом. Стоит отметить, что этот приём часто помогает выигрывать различные соревнования по анализу данных на Kaggle, однако не всегда применим в реальных приложениях.

# Ансамбли в контексте проблемы bias-variance

Вспомним, что в машинном обучении ошибку модели можно разложить на три составляющие:
* смещение (bias);
* разброс (variance);
* неконтролируемая ошибка.

В идеале мы бы хотели, чтобы наша модель имела малый разброс (попадала "точно в цель") и малое смещение (результаты были устойчивыми к изменению входных данных). Но в жизни всё не так просто. Модели с очень малым смещением имеют, как правило, большой разброс (слишком сложные и переобученные), а модели с низким разбросом, в свою очередь, являются слишком простыми (недообученными) и имеют большое смещение. Задача инженера по машинному обучению --- найти компромиссную по сложности модель, или "золотую середину" между разбросом и смещением (bias-variance tradeoff).

![](https://neurohive.io/wp-content/uploads/2019/04/1_kISLC1Udq0m6g5kwHhMuJg-2x-770x487.png)

"Плохие модели" (или, как их ещё называют, "слабые ученики") имеют либо слишком большое смещение, либо слишком большой разброс. Различные типы ансамблей призваны сгладить эти недостатки. Разберёмся подробнее.

Одним из важных моментов является то, что наш выбор слабых учеников должен быть согласован с тем, как мы агрегируем эти модели. 
* Если мы выбираем слабых учеников с **низким смещением, но высоким разбросом**, это должно быть с помощью метода агрегирования, который имеет тенденцию **уменьшать разброс**, тогда как 
* Если мы выбираем слабых учеников с **низким разбросом, но с высоким смещением**, это должен быть метод агрегирования, который имеет тенденцию **уменьшать смещение**.

В контексте рассмотренных нами видов ансамблей:
* В **бэггинге** рассматриваются базовые модели с **низким смещением, но высоким разбросом** (например, переобученные деревья). Усреднение в бустинге будет уменьшать разброс.
* В **бустинге и стекинге** рассматриваются базовые модели с **низким разбросом** (т. е. простые, "глупые"), но **высоким смещением** (например, бустинг часто начинают с неглубоких деревьев). Последовательное улучшение будет давать модели со всё меньшим смещением, чем исходная.

# Полезные ссылки

1. [Ансамблевые методы: бэггинг, бустинг и стекинг](https://neurohive.io/ru/osnovy-data-science/ansamblevye-metody-begging-busting-i-steking/)

2. [Градиентный бустинг](https://habr.com/ru/company/ods/blog/327250/)

3. [Cтекинг и блендинг](https://dyakonov.org/2017/03/10/c%D1%82%D0%B5%D0%BA%D0%B8%D0%BD%D0%B3-stacking-%D0%B8-%D0%B1%D0%BB%D0%B5%D0%BD%D0%B4%D0%B8%D0%BD%D0%B3-blending/)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Бустинг в действии

В модуле [ensemble](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.ensemble) библиотеки sklearn реализовано два основных типа бустинговых ансамблей:
* AdaBoost (адаптивный бустинг);
* GradientBoosting (градиентный бустинг).

Также существуют другие популярные библиотеки, такие как 
* [LightGBM](https://lightgbm.readthedocs.io/en/latest/);
* [XGBoost](https://xgboost.readthedocs.io/en/latest/).

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix, precision_score, recall_score, f1_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt

In [ ]:
def print_results(y_true, y_pred):
    print(confusion_matrix(y_true, y_pred))
    print('F1-score:', f1_score(y_true, y_pred))

In [ ]:
def plot_validation_curve(model_grid, param_name, params=None):
    # Рисуем валидационную кривую
    # По оси х --- значения гиперпараметров (param_***)
    # По оси y --- значения метрики (mean_test_score)

    results_df = pd.DataFrame(model_grid.cv_results_)
    
    if params == None:
        plt.plot(results_df['param_'+param_name], results_df['mean_test_score'])
    else:
        plt.plot(params, results_df['mean_test_score'])

    # Подписываем оси и график
    plt.xlabel(param_name)
    plt.ylabel('Test F1 score')
    plt.title('Validation curve')
    plt.show()

In [ ]:
df = pd.read_csv('/kaggle/input/depression/b_depressed.csv')
df.head()

In [ ]:
# Удалим пропуски
df_1 = df.dropna()

# Дропнем ненужные столбцы
df_2 = df_1.drop(['Survey_id', 'depressed'], axis=1)

# Переведём признаки "Номер виллы" и "Уровень образования" в бинарные 
# * мы не уверены на 100 %, что уровень образования ранговый, поэтому считаем его категориальным
df_3 = pd.get_dummies(df_2, columns=['Ville_id', 'education_level'])
df_3.head()

In [ ]:
# Разделение на train и valid
X = df_3
y = df_1['depressed']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25,
                                                      random_state=19)

In [ ]:
# Масштабирование
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_valid = scaler.transform(X_valid)

In [ ]:
ab = AdaBoostClassifier(random_state=19)
ab.fit(X_train, y_train)
y_pred = ab.predict(X_valid)
print_results(y_valid, y_pred)

In [ ]:
# Тюнинг параметров
ab_n_estimators = {'n_estimators': np.arange(10, 100, 10)}
ab_grid = GridSearchCV(ab, ab_n_estimators, cv=5, scoring='f1', n_jobs=-1)
ab_grid.fit(X_train, y_train)

print(ab_grid.best_score_)
print(ab_grid.best_params_)

In [ ]:
plot_validation_curve(ab_grid, 'n_estimators')

In [ ]:
ab_n_estimators = {'n_estimators': np.arange(100, 201, 20)}
ab_grid = GridSearchCV(ab, ab_n_estimators, cv=5, scoring='f1', n_jobs=-1)
ab_grid.fit(X_train, y_train)

print(ab_grid.best_score_)
print(ab_grid.best_params_)

In [ ]:
plot_validation_curve(ab_grid, 'n_estimators')

In [ ]:
ab_best = ab_grid.best_estimator_
y_pred = ab_best.predict(X_valid)
print_results(y_valid, y_pred)

In [ ]:
ab_l_rate = {'learning_rate': [0.001, 0.01, 0.1, 0.5, 0.75, 1, 2, 10]}
ab_grid = GridSearchCV(ab, ab_l_rate, cv=5, scoring='f1', n_jobs=-1)
ab_grid.fit(X_train, y_train)

print(ab_grid.best_score_)
print(ab_grid.best_params_)

In [ ]:
plot_validation_curve(ab_grid, 'learning_rate')

In [ ]:
ab_best = ab_grid.best_estimator_
y_pred = ab_best.predict(X_valid)
print_results(y_valid, y_pred)

In [ ]:
ab_params = {'n_estimators': np.arange(20, 201, 20), 
             'learning_rate': [0.001, 0.01, 0.1, 0.5, 0.75, 1, 2, 5, 10]}
ab_grid = GridSearchCV(ab, ab_params, cv=5, scoring='f1', n_jobs=-1)
ab_grid.fit(X_train, y_train)

print(ab_grid.best_score_)
print(ab_grid.best_params_)

In [ ]:
ab_best = ab_grid.best_estimator_
y_pred = ab_best.predict(X_valid)
print_results(y_valid, y_pred)

In [ ]:
gb = GradientBoostingClassifier(random_state=19)
gb.fit(X_train, y_train)
y_pred = gb.predict(X_valid)
print_results(y_valid, y_pred)

In [ ]:
gb_n_estimators = {'n_estimators': np.arange(20, 201, 20)}
gb_grid = GridSearchCV(gb, gb_n_estimators, cv=5, scoring='f1', n_jobs=-1)
gb_grid.fit(X_train, y_train)

print(gb_grid.best_score_)
print(gb_grid.best_params_)

In [ ]:
gb_best = gb_grid.best_estimator_
y_pred = gb_best.predict(X_valid)
print_results(y_valid, y_pred)

In [ ]:
gb_l_rate = {'learning_rate': [0.05, 0.1, 0.5, 0.75, 1, 2, 5, 10]}
gb_grid = GridSearchCV(gb, gb_l_rate, cv=5, scoring='f1', n_jobs=-1)
gb_grid.fit(X_train, y_train)

print(gb_grid.best_score_)
print(gb_grid.best_params_)

In [ ]:
gb_best = gb_grid.best_estimator_
y_pred = gb_best.predict(X_valid)
print_results(y_valid, y_pred)

In [ ]:
# gb_params = {'n_estimators': np.arange(20, 201, 20), 
#              'learning_rate': [0.1, 0.5, 0.75, 1, 2, 5, 10]}
# gb_grid = GridSearchCV(gb, gb_params, cv=5, scoring='f1', n_jobs=-1)
# gb_grid.fit(X_train, y_train)

# print(gb_grid.best_score_)
# print(gb_grid.best_params_)

In [ ]:
# gb_best = gb_grid.best_estimator_
# y_pred = gb_best.predict(X_valid)
# print_results(y_valid, y_pred)

In [ ]:
gb_max_depth = {'max_depth': np.arange(1, 11)}
gb_grid = GridSearchCV(gb, gb_max_depth, cv=5, scoring='f1', n_jobs=-1)
gb_grid.fit(X_train, y_train)

print(gb_grid.best_score_)
print(gb_grid.best_params_)

In [ ]:
plot_validation_curve(gb_grid, 'max_depth')

In [ ]:
gb_best = gb_grid.best_estimator_
y_pred = gb_best.predict(X_valid)
print_results(y_valid, y_pred)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_valid, y_pred)

In [ ]:
# gb_params = {'n_estimators': np.arange(20, 221, 50), 
#              'learning_rate': [0.1, 0.5, 1, 5, 10],
#              'max_depth': np.arange(1, 11, 2)}
# gb_grid = GridSearchCV(gb, gb_params, cv=5, scoring='roc_auc', n_jobs=-1)
# gb_grid.fit(X_train, y_train)

# print(gb_grid.best_score_)
# print(gb_grid.best_params_)

In [ ]:
# gb_best = gb_grid.best_estimator_
# y_pred = gb_best.predict(X_valid)
# print_results(y_valid, y_pred)

In [ ]:
import xgboost as xgb
xgbc = xgb.XGBClassifier()
xgbc.fit(X_train, y_train)
y_pred = xgbc.predict(X_valid)
print_results(y_valid, y_pred)

In [ ]:
xgb_params = {'n_estimators': [20, 50, 100, 200],
             'max_depth': [2, 4, 6]}
xgb_grid = GridSearchCV(xgbc, xgb_params, cv=5, scoring='roc_auc', n_jobs=-1)
xgb_grid.fit(X_train, y_train)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

In [ ]:
xgb_best = xgb_grid.best_estimator_
y_pred = xgb_best.predict(X_valid)
print_results(y_valid, y_pred)